In [14]:
#!pip uninstall -y google
!pip install -q google==2.0.3
!pip install -q requests-html==0.10.0


In [2]:
## Load the libraries
from googlesearch import search_news
from googlesearch import search
import pandas as pd
import time
import random
import requests
from requests_html import HTMLSession


In [3]:
## Define/group functions to be used below
def random_agent():
  user_agent_list = [
      'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
      'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
      'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
      "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"
      ]
  for _ in user_agent_list:
    #Pick a random user agent
    user_agent = random.choice(user_agent_list)
  return {'User-Agent': user_agent}



def get_relevant_url2(query, num_result):
    ## This got the api from google
    #url = search_news(query=query, tld="com", lang="en", stop=num_result, pause=2)
    url = search(query=query, tld="com", lang="en", stop=num_result, pause=2)
    return pd.DataFrame({'url':url})



def parse_elements(url,element):
    session = HTMLSession()
    r = session.get(url)
    elements = r.html.find(element)
    return elements


def url_status(url):
  r = requests.get(url=url, headers=random_agent())
  return r.status_code


def add_content(df,colname):
  df[colname] = df['url'].apply(lambda x: parse_elements(x,colname))
  return df


def extract_text(pelement):
  p_list = []
  for pp in range(0,len(pelement)):
    ptext = pelement[pp].text
    if len(ptext) > 50:
      p_list.append( ptext )
  return ' '.join(p_list)


def search_statement_get_context(statement, num_links=1):

    ## Get links from google search
    df = get_relevant_url2(statement,num_links)
    df['status_code'] = df['url'].apply(lambda x: url_status(x))

    ## Search link with successful returns
    df1 = df.loc[df.status_code==200].reset_index(drop=True)
    print('Got '+str(df1.shape[0])+' out of '+str(df.shape[0])+' valid urls for further searching ... ')
    
    ## Get context from urls
    df1 = add_content(df1,'p') ## Add paragraph
    # df1 = add_content(df1,'a') ## Add link
    # df1 = add_content(df1,'img') ## Add img
    
    ## Concat paragraph into 1 line in cell
    df1['p_text'] = df1['p'].apply( lambda x: extract_text(x) )
    
    return df1


In [8]:
## Testing on statement and fetched context
if False:
    statement = 'Joe Biden said not to vote'
    
    ### Get contexts online
    df = search_statement_get_context(statement,3)

    print(df)


Got 3 out of 3 valid urls for further searching ... 


,url,status_code,p,p_text
0,https://www.forbes.com/sites/tylerroush/2024/0...,200,"[<Element 'p' >, <Element 'p' class=('color-bo...",A robocall of unknown origin featuring a fake ...
1,https://www.nbcnews.com/politics/2024-election...,200,"[<Element 'p' class=('headline-title-text', 'h...","MANCHESTER, N.H. — The New Hampshire attorney ..."
2,https://www.whitehouse.gov/briefing-room/speec...,200,"[<Element 'p' >, <Element 'p' class=('has-text...",The White House\n1600 Pennsylvania Ave NW\nWas...
